In [14]:
import os

import pandas as pd

from utils.optuna_database_manager import OptunaDBManager

db_root_path = "../../data/backtesting/"
db_name = "backtesting_report.db"

optuna_db_manager = OptunaDBManager(db_name=db_name,
                                    db_root_path=db_root_path)

In [16]:
optuna_db_manager.studies

,study_id,study_name
0,1,super_trend_optimization_1


In [32]:
study_name = "super_trend_optimization_1"
df = optuna_db_manager.merged_df[optuna_db_manager.merged_df["study_name"] == study_name]

In [33]:
df

,trial_id,number,study_id,state,datetime_start,datetime_complete,study_name,accuracy,avg_trading_time_in_hours,config,...,total_positions,win_signals,trial_value_id,objective_x,value,value_type,study_direction_id,direction,objective_y,hover_text
0,1,0,1,COMPLETE,2023-12-07 16:34:08.257012,2023-12-07 16:34:11.216496,super_trend_optimization_1,0.224806,4.111413,"""{\""exchange\"": \""binance_perpetual\"", \""tradi...",...,129,29,1,0,0.008787,FINITE,1,MAXIMIZE,0,<b>Trial ID: 1</b><br><b>Study: super_trend_op...
1,2,1,1,COMPLETE,2023-12-07 16:34:11.226266,2023-12-07 16:34:17.906535,super_trend_optimization_1,0.314465,2.170325,"""{\""exchange\"": \""binance_perpetual\"", \""tradi...",...,318,100,2,0,0.291448,FINITE,1,MAXIMIZE,0,<b>Trial ID: 2</b><br><b>Study: super_trend_op...
2,3,2,1,COMPLETE,2023-12-07 16:34:17.914268,2023-12-07 16:34:19.421124,super_trend_optimization_1,0.351351,6.009647,"""{\""exchange\"": \""binance_perpetual\"", \""tradi...",...,74,26,3,0,-0.085482,FINITE,1,MAXIMIZE,0,<b>Trial ID: 3</b><br><b>Study: super_trend_op...
3,4,3,1,COMPLETE,2023-12-07 16:34:19.428529,2023-12-07 16:34:27.061663,super_trend_optimization_1,0.499397,0.773337,"""{\""exchange\"": \""binance_perpetual\"", \""tradi...",...,829,414,4,0,-0.141714,FINITE,1,MAXIMIZE,0,<b>Trial ID: 4</b><br><b>Study: super_trend_op...
4,5,4,1,COMPLETE,2023-12-07 16:34:27.069613,2023-12-07 16:34:34.241515,super_trend_optimization_1,0.352941,3.923623,"""{\""exchange\"": \""binance_perpetual\"", \""tradi...",...,187,66,5,0,-0.223673,FINITE,1,MAXIMIZE,0,<b>Trial ID: 5</b><br><b>Study: super_trend_op...
5,6,5,1,COMPLETE,2023-12-07 16:34:34.249291,2023-12-07 16:34:38.893255,super_trend_optimization_1,0.510753,2.671102,"""{\""exchange\"": \""binance_perpetual\"", \""tradi...",...,186,95,6,0,-0.045817,FINITE,1,MAXIMIZE,0,<b>Trial ID: 6</b><br><b>Study: super_trend_op...
6,7,6,1,COMPLETE,2023-12-07 16:34:38.900783,2023-12-07 16:34:41.791808,super_trend_optimization_1,0.505415,1.081769,"""{\""exchange\"": \""binance_perpetual\"", \""tradi...",...,277,140,7,0,-0.071774,FINITE,1,MAXIMIZE,0,<b>Trial ID: 7</b><br><b>Study: super_trend_op...
7,8,7,1,COMPLETE,2023-12-07 16:34:41.799046,2023-12-07 16:34:47.384110,super_trend_optimization_1,0.364238,4.141523,"""{\""exchange\"": \""binance_perpetual\"", \""tradi...",...,151,55,8,0,0.012283,FINITE,1,MAXIMIZE,0,<b>Trial ID: 8</b><br><b>Study: super_trend_op...
8,9,8,1,COMPLETE,2023-12-07 16:34:47.391790,2023-12-07 16:34:51.932509,super_trend_optimization_1,0.375000,4.488008,"""{\""exchange\"": \""binance_perpetual\"", \""tradi...",...,128,48,9,0,0.085992,FINITE,1,MAXIMIZE,0,<b>Trial ID: 9</b><br><b>Study: super_trend_op...
9,10,9,1,COMPLETE,2023-12-07 16:34:51.940085,2023-12-07 16:34:58.022508,super_trend_optimization_1,0.300813,5.096682,"""{\""exchange\"": \""binance_perpetual\"", \""tradi...",...,123,37,10,0,-0.266434,FINITE,1,MAXIMIZE,0,<b>Trial ID: 10</b><br><b>Study: super_trend_o...


### Bar Chart of Average Trading Time
This bar chart compares the average trading time across trials. It helps to quickly identify trials with unusually long or short trading times.


In [35]:
import plotly.express as px

fig = px.bar(df, x='trial_id', y='avg_trading_time_in_hours', title='Average Trading Time per Trial')
fig.show()

### Scatter Plot of Accuracy vs. Average Trading Time
This scatter plot shows the relationship between the accuracy and the average trading time of each trial. It can help to identify if there is any correlation between these two metrics.


In [36]:
fig = px.scatter(df, x='avg_trading_time_in_hours', y='accuracy', title='Accuracy vs. Average Trading Time')
fig.show()


### Histogram of Total Positions
The histogram represents the distribution of total positions across all trials. This visualization is useful for understanding the general spread and most common values of positions.


In [37]:
fig = px.histogram(df, x='total_positions', title='Distribution of Total Positions')
fig.show()

### Pie Chart of Win Signals
This pie chart shows the proportion of win signals in each trial, providing a visual representation of the success rate distribution across trials.


In [39]:
fig = px.pie(df, names='trial_id', values='win_signals', title='Proportion of Win Signals per Trial')
fig.show()


### Box Plot for Trial Value
A box plot for trial values to identify the range, median, and any potential outliers in the trial values.


In [41]:
fig = px.box(df, y='value', title='Box Plot of Trial PNL')
fig.show()

### Heatmap for Correlation Analysis
This heatmap illustrates the correlation between various numerical variables such as accuracy, average trading time, total positions, win signals, and value.


In [45]:
import plotly.express as px

# Calculate the correlation matrix
correlation_matrix = df[['accuracy', 'avg_trading_time_in_hours', 'total_positions', 'win_signals', 'value']].corr()

# Generate the heatmap
fig = px.imshow(correlation_matrix,
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                title='Correlation Heatmap',
                labels=dict(x="Variable", y="Variable", color="Correlation"),
                color_continuous_scale='RdBu')

fig.show()



### Stacked Bar Chart for Win Signals vs Total Positions
A stacked bar chart displaying the ratio of win signals to total positions for each trial. This helps in visualizing the efficiency and effectiveness of each trial.


In [46]:
df['loss_signals'] = df['total_positions'] - df['win_signals']
fig = px.bar(df, x='trial_id', y=['win_signals', 'loss_signals'], title='Win vs Loss Signals per Trial', labels={'value':'Number of Signals'}, hover_data=['total_positions'])
fig.show()


### Scatter Plots Against PNL
These scatter plots show how various metrics behave in relation to the PNL (Profit and Loss). This analysis can help in understanding which factors have a stronger relationship with financial outcomes.


In [47]:
# Scatter Plot for Accuracy vs PNL
fig_accuracy_pnl = px.scatter(df, x='accuracy', y='value', title='Accuracy vs PNL')
fig_accuracy_pnl.show()

# Scatter Plot for Average Trading Time vs PNL
fig_tradingtime_pnl = px.scatter(df, x='avg_trading_time_in_hours', y='value', title='Average Trading Time vs PNL')
fig_tradingtime_pnl.show()

# Scatter Plot for Total Positions vs PNL
fig_positions_pnl = px.scatter(df, x='total_positions', y='value', title='Total Positions vs PNL')
fig_positions_pnl.show()

# Scatter Plot for Win Signals vs PNL
fig_winsignals_pnl = px.scatter(df, x='win_signals', y='value', title='Win Signals vs PNL')
fig_winsignals_pnl.show()


### Scatter Matrix of All Variables
A scatter matrix allows us to see both the distribution of single variables and the relationships between two variables. The diagonal shows histograms for each variable, and the scatter plots show correlations between them.


In [49]:
import plotly.express as px

# Selecting columns for scatter matrix
selected_columns = ['accuracy', 'avg_trading_time_in_hours', 'total_positions', 'win_signals', 'value']

# Creating scatter matrix
fig_matrix = px.scatter_matrix(df[selected_columns], dimensions=selected_columns, title='Scatter Matrix of Variables', height=800)
fig_matrix.show()
